<br>

# Dados Tabulares

In [ ]:
import os
import py7zr
import requests
import pandas as pd
import geopandas as gpd

from osgeo import ogr
from zipfile import ZipFile

In [ ]:
from paths import *
from functions import *

<br>

## Get Data

In [ ]:
# Input
url = 'https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2020/UFs/SP/SP_Municipios_2020.zip'
filename = 'sp_ibge_250k.zip'

# Get Data
get_download(url, filename, shps_path)
unzip(filename, shps_path)

In [ ]:
# Input
url = 'http://datageo.ambiente.sp.gov.br/geoserver/datageo/LimiteMunicipal/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=LimiteMunicipal'
filename = 'sp_igc_050k.zip'

# Get Data
get_download(url, filename, shps_path)
unzip(filename, shps_path)

<br>

## Área

In [ ]:
# Converte em *geojson*
file = 'sp_ibge_250k'
shp = os.path.join(shps_path, '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Deleta colunas
gdf.drop([
    'NM_MUN',
    'SIGLA_UF',
],
    axis=1,
    inplace=True
)

# Renomeia colunas
gdf.rename(
    columns={
        'CD_MUN': 'id_municipio',
        'AREA_KM2': 'area_km2',
    },
    inplace=True
)

# Ajusta Tipo
gdf['id_municipio'] = gdf['id_municipio'].astype(int)

# Create Dataframe
df = gdf[['id_municipio', 'area_km2']]

# Salva
df.to_csv(
    os.path.join(tabs_path, 'tab_municipio_area.csv'),
    index=False,
)

# Resultados
df.head()

<br>

## CETESB

In [ ]:
# Lê arquivo
file = 'sp_igc_050k'
shp = os.path.join(shps_path, '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Cria Cópia
df = gdf

# Renomeia colunas
df.rename(
    columns={
        'Cod_ibge'  : 'id_municipio',
        'Cod_Cetesb': 'id_cetesb',
    },
    inplace=True
)

# Seleciona
df = df[['id_municipio', 'id_cetesb']].copy()

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    os.path.join(tabs_path, 'tab_municipio_nome.csv'),
    usecols=['id_municipio', 'municipio_nome'],
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio',
)

# Salva
df.to_csv(
    os.path.join(tabs_path, 'tab_municipio_idcetesb.csv'),
    index=False,
)

# Resultados
df.head()

<br>

## UGRHIs

In [ ]:
def rename_nome(x):
    x = x.title()
    x = x.strip()
    dict_rename = {
        # Encoding
        '/': ' / ',
        '  '  : ' ',
        ' De '  : ' de ',
        ' Da '  : ' da ',
        ' Do '  : ' do ',
        ' Dos '  : ' dos ',
        'Jundiai'  : 'Jundiaí',        
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

In [ ]:
# Lê arquivo
file = 'sp_igc_050k'
shp = os.path.join(shps_path, '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Cria Cópia
df = gdf

# Renomeia colunas
df.rename(
    columns={
        'Cod_ibge'  : 'id_municipio',
        'Nome_ugrhi': 'nome_ugrhi',
        'UGRHI'     : 'id_ugrhi',
    },
    inplace=True
)

# Seleciona
df = df[['id_municipio', 'id_ugrhi', 'nome_ugrhi']].copy()

# Renomeia Campos Coluna
df.loc[:, 'nome_ugrhi'] = df['nome_ugrhi'].astype(str).apply(lambda x: rename_nome(x))

# Resultados
df.head()

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    os.path.join(tabs_path, 'tab_municipio_nome.csv'),
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Salva
df.to_csv(
    os.path.join(tabs_path, 'tab_municipio_ugrhi.csv'),
    index=False,
)

# Resultados
df.head()

<br>

## RMs e AUs

In [ ]:
# URL para baixar o arquivo zip
url = 'https://geoftp.ibge.gov.br/organizacao_do_territorio/estrutura_territorial/municipios_por_regioes_metropolitanas/Situacao_2020a2029/Composicao_RMs_RIDEs_AglomUrbanas_2020_06_30.xlsx'

# Read File
df = pd.read_excel(
    url,
    engine='openpyxl',
)
print('Existem as seguintes colunas disponíveis:\n{}\n'.format(list(df.columns)))

df = df.loc[df['COD_UF'] == 35]
df = df[['COD_MUN', 'NOME', 'LEG', 'DATA']].copy()
df.reset_index(drop=True, inplace=True)

df.rename(
    columns={
        'COD_MUN': 'id_municipio',
        'NOME'   : 'nome_rm',
        'LEG'    : 'legislacao',
        'DATA'   : 'data',
    },
    inplace=True,
)

# Atribute Table
df['data_format'] = df['data'].dt.strftime('%d.%m.%Y')
df['legislacao_data'] = df[['legislacao', 'data_format']].apply(lambda x: ' de '.join(x), axis=1)
df.drop('data_format', axis=1, inplace=True)

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    os.path.join(tabs_path, 'tab_municipio_nome.csv'),
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='right',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Results
df.to_csv(
    os.path.join(tabs_path, 'tab_rms.csv'),
    index=False,
)

# Resultados
df.head()

<br>

## Microregião

In [ ]:
# Lê geojson
file = 'sp_ibge_250k'
shp = os.path.join(shps_path, '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Cria Cópia
df = gdf[['CD_MUN']].copy()

# Renomeia colunas
df.rename(columns={'CD_MUN'  : 'id_municipio'}, inplace=True)

# Pega apenas os três primeiros caracteres de um campo de número de maneira mais eficiente
df['id_microregiao'] = [int(x[0:5]) for x in df['id_municipio']]

# Ajusta Tipo
df['id_municipio'] = df['id_municipio'].astype(int)

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    os.path.join(tabs_path, 'tab_municipio_nome.csv'),
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Salva
df.to_csv(
    os.path.join(tabs_path, 'tab_municipio_microregiao.csv'),
    index=False,
)

# Resultados
df.head()

<br>

## Mesoregião

In [ ]:
# Lê geojson
file = 'sp_ibge_250k'
shp = os.path.join(shps_path, '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Cria Cópia
df = gdf[['CD_MUN']].copy()

# Renomeia colunas
df.rename(columns={'CD_MUN'  : 'id_municipio'}, inplace=True)

# Pega apenas os três primeiros caracteres de um campo de número de maneira mais eficiente
df['id_mesoregiao'] = [int(x[0:4]) for x in df['id_municipio']]

# Ajusta Tipo
df['id_municipio'] = df['id_municipio'].astype(int)

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    os.path.join(tabs_path, 'tab_municipio_nome.csv'),
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Salva
df.to_csv(
    os.path.join(tabs_path, 'tab_municipio_mesoregiao.csv'),
    index=False,
)

# Resultados
df.head()

<br>

## Macrometropole

In [ ]:
# Input
url = 'http://datageo.ambiente.sp.gov.br/geoserver/datageo/LimiteMacrometropole2015/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=LimiteMacrometropole2015'
filename = 'sp_macrometropole.zip'

# Get Data
get_download(url, filename, shps_path)
unzip(filename, shps_path)

In [ ]:
# Lê arquivo
file = 'sp_macrometropole'
shp = os.path.join(shps_path, '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Renomeia colunas
gdf.rename(columns={'Nome': 'macrometropole'}, inplace=True)

# Results
display(gdf.head())

# Cria Cópia
gdf_macro = gdf

# Deleta Shapefile
delete_shapefile(shp)

In [ ]:
# Lê arquivo
file = 'sp_igc_050k'
shp = os.path.join(shps_path, '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Renomeia colunas
gdf.rename(
    columns={
        'Cod_ibge'  : 'id_municipio',
        'Cod_Cetesb': 'id_cetesb',
        'Nome'      : 'nome_municipio',
        'Rotulo'    : 'rotulo_municipio',
        'Nome_ugrhi': 'nome_ugrhi',
        'UGRHI'     : 'id_ugrhi',
    },
    inplace=True
)

# Deleta colunas
gdf.drop(['id_cetesb', 'nome_municipio', 'rotulo_municipio', 'nome_ugrhi', 'id_ugrhi'], axis=1, inplace=True)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Pega centroid
gdf_geometry = gdf.representative_point()
gdf['geometry'] = gdf_geometry

# Results
display(gdf.head())
gdf.plot()

In [ ]:
# Intersect
gdf_inter = gpd.sjoin(gdf, gdf_macro, how='inner', op='intersects')

# Convert to Dataframe
df = gdf_inter[['id_municipio', 'macrometropole']].copy()

# Results
display(df.head())
gdf_inter.plot()

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    os.path.join(tabs_path, 'tab_municipio_nome.csv'),
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='right',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Salva
df.to_csv(
    os.path.join(tabs_path, 'tab_macrometropole.csv'),
    index=False,
)

# Resultados
df.head()

<br>

## Regiões do Governo

In [ ]:
# Input
url = 'http://datageo.ambiente.sp.gov.br/geoserver/datageo/RegioesGoverno/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=RegioesGoverno'
filename = 'sp_regioesgoverno.zip'

# Get Data
get_download(url, filename, shps_path)
unzip(filename, shps_path)

In [ ]:
def rename_nome(x):
    x = x.title()
    x = x.strip()
    dict_rename = {
        '  '  : ' ',
        ' De '  : ' de ',
        ' Da '  : ' da ',
        ' Do '  : ' do ',
        ' Dos '  : ' dos ',        
        ' Jose '  : ' José ',
        ' Joao '  : ' João ',
        'Jau'  : 'Jaú',
        'Sao ': 'São ',
        'Avare': 'Avaré',
        'Jundiai': 'Jundiaí',
        'Marilia': 'Marília',
        'Taubate': 'Taubaté',
        'Ribeirao': 'Ribeirão',
        'Guaratingueta': 'Guaratinguetá',
        'Fernandopolis': 'Fernandópolis',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

In [ ]:
# Lê arquivo
file = 'sp_regioesgoverno'
shp = os.path.join(shps_path, '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Renomeia colunas
gdf.rename(columns={'Nome': 'regiaogoverno_nome'}, inplace=True)

# Renomeia Campos Coluna
gdf.loc[:, 'regiaogoverno_nome'] = gdf['regiaogoverno_nome'].astype(str).apply(lambda x: rename_nome(x))

# Results
display(gdf.head())

# Cria Cópia
gdf_gov = gdf

# Deleta Shapefile
delete_shapefile(shp)

In [ ]:
# Lê arquivo
file = 'sp_igc_050k'
shp = os.path.join(shps_path, '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Renomeia colunas
gdf.rename(
    columns={
        'Cod_ibge'  : 'id_municipio',
        'Cod_Cetesb': 'id_cetesb',
        'Nome'      : 'nome_municipio',
        'Rotulo'    : 'rotulo_municipio',
        'Nome_ugrhi': 'nome_ugrhi',
        'UGRHI'     : 'id_ugrhi',
    },
    inplace=True
)

# Deleta colunas
gdf.drop(['id_cetesb', 'nome_municipio', 'rotulo_municipio', 'nome_ugrhi', 'id_ugrhi'], axis=1, inplace=True)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Pega centroid
gdf_geometry = gdf.representative_point()
gdf['geometry'] = gdf_geometry

# Results
display(gdf.head())
gdf.plot()

In [ ]:
# Intersect
gdf_inter = gpd.sjoin(gdf, gdf_gov, how='inner', op='intersects')

# Convert to Dataframe
df = gdf_inter[['id_municipio', 'regiaogoverno_nome']].copy()

# Results
display(df.head())
gdf_inter.plot()

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    os.path.join(tabs_path, 'tab_municipio_nome.csv'),
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Salva
df.to_csv(
    os.path.join(tabs_path, 'tab_municipio_regiaogoverno.csv'),
    index=False,
)

# Resultados
df.head()

<br>

## Regiões Administrativas

In [ ]:
# Input
url = 'http://datageo.ambiente.sp.gov.br/geoserver/datageo/RegiaoAdministrativa/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=RegiaoAdministrativa'
filename = 'sp_regioesadm.zip'

# Get Data
get_download(url, filename, shps_path)
unzip(filename, shps_path)

In [ ]:
def rename_nome(x):
    x = x.title()
    x = x.strip()
    dict_rename = {
        '  '  : ' ',
        ' De '  : ' de ',
        ' Da '  : ' da ',
        ' Do '  : ' do ',
        ' Dos '  : ' dos ',        
        ' Jose '  : ' José ',
        ' Joao '  : ' João ',
        'Jau'  : 'Jaú',
        'Sao ': 'São ',
        'Avare': 'Avaré',
        'Jundiai': 'Jundiaí',
        'Marilia': 'Marília',
        'Taubate': 'Taubaté',
        'Ribeirao': 'Ribeirão',
        'Guaratingueta': 'Guaratinguetá',
        'Fernandopolis': 'Fernandópolis',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

In [ ]:
# Lê arquivo
file = 'sp_regioesadm'
shp = os.path.join(shps_path, '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Renomeia colunas
gdf.rename(columns={'Nome': 'regiaoadm_nome'}, inplace=True)

# Renomeia Campos Coluna
gdf.loc[:, 'regiaoadm_nome'] = gdf['regiaoadm_nome'].astype(str).apply(lambda x: rename_nome(x))

# Results
display(gdf.head())

# Cria Cópia
gdf_adm = gdf

# Deleta Shapefile
delete_shapefile(shp)

In [ ]:
# Lê arquivo
file = 'sp_igc_050k'
shp = os.path.join(shps_path, '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Renomeia colunas
gdf.rename(
    columns={
        'Cod_ibge'  : 'id_municipio',
        'Cod_Cetesb': 'id_cetesb',
        'Nome'      : 'nome_municipio',
        'Rotulo'    : 'rotulo_municipio',
        'Nome_ugrhi': 'nome_ugrhi',
        'UGRHI'     : 'id_ugrhi',
    },
    inplace=True
)

# Deleta colunas
gdf.drop(['id_cetesb', 'nome_municipio', 'rotulo_municipio', 'nome_ugrhi', 'id_ugrhi'], axis=1, inplace=True)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Pega centroid
gdf_geometry = gdf.representative_point()
gdf['geometry'] = gdf_geometry

# Results
display(gdf.head())
gdf.plot()

In [ ]:
# Intersect
gdf_inter = gpd.sjoin(gdf, gdf_adm, how='inner', op='intersects')

# Convert to Dataframe
df = gdf_inter[['id_municipio', 'regiaoadm_nome']].copy()

# Results
display(df.head())
gdf_inter.plot()

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    os.path.join(tabs_path, 'tab_municipio_nome.csv'),
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Salva
df.to_csv(
    os.path.join(tabs_path, 'tab_municipio_regiaoadm.csv'),
    index=False,
)

# Resultados
df.head()

<br>

# Delete Temp Files

In [ ]:
# Deleta Shapefile
file = 'sp_ibge_250k'
shp = os.path.join(shps_path, '{}.shp'.format(file))
delete_shapefile(shp)

In [ ]:
# Deleta Shapefile
file = 'sp_igc_050k'
shp = os.path.join(shps_path, '{}.shp'.format(file))
delete_shapefile(shp)

In [ ]:
import shutil
shutil.rmtree(shps_path, ignore_errors=True, onerror=print('Erro'))